In [1]:
# Version 12
# Manual loading images, image augmentation - image data generator
# change : padding in generator change to "cval, black"
# Conclusions : very small difference and we get 98-100%  :)

In [2]:
#all imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import csv
import imageio

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K
from keras.optimizers import SGD
from keras import regularizers


Using TensorFlow backend.


In [3]:
K.set_image_dim_ordering('th')
seed = 7
np.random.seed(seed)

In [4]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def readimagesfromfolder(path, color, show):
	imagepackage_list = []
	imagepackage = []
	imagelist = []
	filelist = sorted(os.listdir(path))
	for filename in filelist:
		fullfilename =  path + filename
		#print(fullfilename)        
		singleimage = imageio.imread(fullfilename)
		singleimage = cv2.resize(singleimage,(96,96), interpolation=cv2.INTER_AREA)        
		singleimage = rgb2gray(singleimage)
		#print(singleimage)        
		if show==1:
			plt.imshow(singleimage)
			plt.show()
		imagepackage.append(singleimage)
		imagepackage_list.append(imagepackage)        
		imagepackage = []        
	#print(len(imagepackage))        
	imagepackage_list = np.float32(imagepackage_list)
	#cv2.destroyAllWindows()
	return imagepackage_list

def readlabelfromcsv(filename):
    training_y = []
    with open(filename) as f:
        #training_y.append(f.readline())
        content = csv.reader(f)
        for line in content:
            #print(line[0])
            training_y.append(int(line[0]))
    return training_y




In [5]:
       
folderpath = 'data/basic/training/'
X_train=readimagesfromfolder(folderpath, 0, 0)
print(np.shape(X_train))

folderpath = 'data/basic/test/'
X_test=readimagesfromfolder(folderpath, 0, 0)
print(np.shape(X_test))

y_train = readlabelfromcsv("data/basic/training.csv")
y_test = readlabelfromcsv("data/basic/test.csv")

print(y_train)
print(np.shape(y_train))

print(y_test)
print(np.shape(y_test))

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,
    fill_mode="constant", 
    cval=0) # randomly flip images) # randomly flip images

datagen.fit(X_train)

(162, 1, 96, 96)
(48, 1, 96, 96)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
(162,)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
(48,)


In [6]:


# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [7]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Convolution2D(16, 3, 1, border_mode='same', input_shape=(1, 96, 96), activation='relu'))
	model.add(Convolution2D(24, 5, 3, border_mode='same', activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Convolution2D(32, 5, 3, border_mode='same', activation='relu'))
	model.add(Convolution2D(48, 5, 3, border_mode='same', activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(1000, activation='relu'))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# build the model
model = larger_model()
# Fit the model
model.fit_generator(datagen.flow(X_train, y_train, batch_size=25),samples_per_epoch = 5000, nb_epoch = 50, verbose=2, validation_data=(X_test, y_test))
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print scores
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Epoch 1/50


/usr/local/lib/python2.7/dist-packages/keras/engine/training.py:1462: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


46s - loss: 0.8391 - acc: 0.6275 - val_loss: 0.6029 - val_acc: 0.7708
Epoch 2/50
20s - loss: 0.4474 - acc: 0.8190 - val_loss: 0.5210 - val_acc: 0.7917
Epoch 3/50
20s - loss: 0.2346 - acc: 0.9110 - val_loss: 0.3155 - val_acc: 0.8542
Epoch 4/50
20s - loss: 0.1094 - acc: 0.9600 - val_loss: 0.3710 - val_acc: 0.8750
Epoch 5/50
20s - loss: 0.0984 - acc: 0.9667 - val_loss: 0.2227 - val_acc: 0.9375
Epoch 6/50
20s - loss: 0.0514 - acc: 0.9827 - val_loss: 0.2390 - val_acc: 0.9375
Epoch 7/50
20s - loss: 0.0408 - acc: 0.9859 - val_loss: 0.1441 - val_acc: 0.9167
Epoch 8/50
20s - loss: 0.0323 - acc: 0.9898 - val_loss: 0.5342 - val_acc: 0.8958
Epoch 9/50
20s - loss: 0.0185 - acc: 0.9932 - val_loss: 0.3233 - val_acc: 0.9375
Epoch 10/50
20s - loss: 0.0433 - acc: 0.9837 - val_loss: 0.2155 - val_acc: 0.8958
Epoch 11/50
20s - loss: 0.0281 - acc: 0.9920 - val_loss: 0.3113 - val_acc: 0.8958
Epoch 12/50
20s - loss: 0.0164 - acc: 0.9948 - val_loss: 0.5240 - val_acc: 0.9167
Epoch 13/50
20s - loss: 0.0226 - acc